In [1]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "Harrison works at LangChain",
        "Karen works at Snowflake",
        "Gor works at Microsoft",
        "Habet works at AUA",
        "Bears like to eat honey",
    ],
    embedding=OpenAIEmbeddings(),
)

/Users/kjavadyan/projects/PyData-March-Langchain/venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [3]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("Who likes to eat honey?")

[Document(page_content='Bears like to eat honey'),
 Document(page_content='Habet works at AUA')]

In [4]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Please answer "I don't know" if the context is not enough to answer a question.
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(temperature=0)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
)
chain = setup_and_retrieval | prompt | model | output_parser

In [5]:
chain.invoke("where is Gor working?")

'Gor is working at Microsoft.'

<img src="image.svg"/>